## Support vector machines

Try with support vector machines in a "Support vector machines.ipynb" notebook. Again, **use the OverFeat features.**

**Q:** Create an SVM classifier with a linear kernel. Tune its **C Parameter**.
There are many ways to create an SVM classifier, e.g., LinearSVC, SVC, or even SGDClassifier with hinge loss. Briefly explain in your code your choice of estimator in both cases.


**A:** As proposed in the course context, SVC is commonly used for nonlinear kernels. As the estimator uses a one.vs.one approach, there will be issues with large numbers of data. On the other hand, the LinearSVC utilized a one.vs.rest approach which is way faster than the one.vs.one while leading to somehow simillar results for linear kernels.

**Note:** You might want to use PCA as a preprocessing step before your decision tree or random forest estimators to improve the results, e.g., speed or accuracy. In any case, justify your choice in a comment or a markdown cell.

**A:** As the PCA filter the principal components in the data and projects the data to the principal axes, it would make the classification much easier and would lead to a decrease in the calculation costs

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

%matplotlib inline

# Load data
with np.load('cifar4-train.npz', allow_pickle=False) as npz_file:
    cifar4 = dict(npz_file.items())

#let's take a look at the keys
print('Data loaded')
print('It is a dictionary with keys:', list(cifar4.keys()))

Data loaded
It is a dictionary with keys: ['pixels', 'overfeat', 'labels', 'names', 'allow_pickle']


In [2]:
# Create X/y arrays
X = cifar4['overfeat']
y = cifar4['labels']

print('X:', X.shape, X.dtype)
print('y:', y.shape, y.dtype)

X: (5000, 4096) float32
y: (5000,) int64


In [3]:
# Split data into train/test sets
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, train_size=4000, test_size=1000, stratify = y, random_state=0)

In [4]:
#let's make sure of the uniform distribution of the data in the train set
pd.value_counts(y_tr, normalize = 'True')

3    0.25
1    0.25
2    0.25
0    0.25
dtype: float64

**Q:** Tune the parameters using **grid search with cross-validation**. Use the stratified 5-fold strategy.

In [5]:
# Create the pipeline and fit it to training data
pca = PCA(n_components = 176) #retain 90% of the variance explained
pipe_LinearSVC = Pipeline([
    ('pca', pca), # The number of components in PCA are adopted from the "Data exploration" section
    ('lsvc', LinearSVC(random_state=0))
])

**Q:** Collect the results in a DataFrame with a column for the mean and the standard deviation of the accuracy values across all folds. For the linear kernel, your DataFrame will have one row for each **C** value.

In [6]:
# let's define the values of parameters for grid search
C_list = np.arange(0.01, 10, 1)

# Grid search with cross-validation
grid = {
    'lsvc__C': C_list
}
grid_cv = GridSearchCV(pipe_LinearSVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
lsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_lsvc__C']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

lsvc_score
# C = 0.01 seems fine

C  mean accuracy  standard deviation
0  0.01        0.82975            0.013657
1  1.01        0.80200            0.009274
2  2.01        0.79050            0.010595
3  3.01        0.77775            0.028344
4  4.01        0.76825            0.013888
5  5.01        0.78025            0.012410
6  6.01        0.76525            0.014414
7  7.01        0.77900            0.010075
8  8.01        0.76375            0.016863
9  9.01        0.75550            0.011927

In [7]:
# let's refine the search range for the C value
C_list = np.arange(0.005, 0.05, 0.005)

# Grid search with cross-validation
grid = {
    'lsvc__C': C_list
}
grid_cv = GridSearchCV(pipe_LinearSVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
lsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_lsvc__C']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

lsvc_score
# C = 0.005 seems fine

C  mean accuracy  standard deviation
0  0.005        0.83100            0.016190
1   0.01        0.83275            0.014778
2  0.015        0.83025            0.013996
3   0.02        0.83025            0.017018
4  0.025        0.83200            0.015137
5   0.03        0.83175            0.014504
6  0.035        0.82900            0.015937
7   0.04        0.82950            0.017349
8  0.045        0.83025            0.015174

In [8]:
# Create the pipeline and fit it to training data
pipe_SVC = Pipeline([
    ('pca', pca), # The number of components in PCA are adopted from the "Data exploration" section
    ('nlsvc', SVC(kernel='rbf', random_state=0))
])

In [9]:
# let's define the values of parameters for grid search
nlC_list = np.arange(0.1, 11, 5)
gamma_list = np.arange(0.001, 1100, 500)

# Grid search with cross-validation
grid = {
    'nlsvc__C': nlC_list,
    'nlsvc__gamma': gamma_list
}
grid_cv = GridSearchCV(pipe_SVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
nlsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_nlsvc__C']),
    ('gamma', grid_cv.cv_results_['param_nlsvc__gamma']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

nlsvc_score
# we can refine the range for gamma

C    gamma  mean accuracy  standard deviation
0   0.1    0.001        0.63950            0.016058
1   0.1  500.001        0.25000            0.000000
2   0.1     1000        0.25000            0.000000
3   5.1    0.001        0.80775            0.003391
4   5.1  500.001        0.25000            0.000000
5   5.1     1000        0.25000            0.000000
6  10.1    0.001        0.80650            0.002669
7  10.1  500.001        0.25000            0.000000
8  10.1     1000        0.25000            0.000000

In [10]:
# let's define the values of parameters for grid search
nlC_list = np.arange(1, 102, 50)
gamma_list = np.arange(0.001, 110, 50)

# Grid search with cross-validation
grid = {
    'nlsvc__C': nlC_list,
    'nlsvc__gamma': gamma_list
}
grid_cv = GridSearchCV(pipe_SVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
nlsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_nlsvc__C']),
    ('gamma', grid_cv.cv_results_['param_nlsvc__gamma']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

nlsvc_score
# we can refine the range for gamma

C    gamma  mean accuracy  standard deviation
0    1    0.001        0.80225            0.005557
1    1   50.001        0.25000            0.000000
2    1  100.001        0.25000            0.000000
3   51    0.001        0.80775            0.003657
4   51   50.001        0.25000            0.000000
5   51  100.001        0.25000            0.000000
6  101    0.001        0.80800            0.002915
7  101   50.001        0.25000            0.000000
8  101  100.001        0.25000            0.000000

In [11]:
# let's define the values of parameters for grid search
nlC_list = np.arange(10, 1100, 500)
gamma_list = np.arange(0.001, 11, 5)

# Grid search with cross-validation
grid = {
    'nlsvc__C': nlC_list,
    'nlsvc__gamma': gamma_list
}
grid_cv = GridSearchCV(pipe_SVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
nlsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_nlsvc__C']),
    ('gamma', grid_cv.cv_results_['param_nlsvc__gamma']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

nlsvc_score
# we can refine the range for gamma

C   gamma  mean accuracy  standard deviation
0    10   0.001        0.80725            0.002894
1    10   5.001        0.25000            0.000000
2    10  10.001        0.25000            0.000000
3   510   0.001        0.80675            0.002318
4   510   5.001        0.25000            0.000000
5   510  10.001        0.25000            0.000000
6  1010   0.001        0.80700            0.003021
7  1010   5.001        0.25000            0.000000
8  1010  10.001        0.25000            0.000000

In [12]:
# let's define the values of parameters for grid search
nlC_list = np.arange(100, 11000, 5000)
gamma_list = np.arange(0.001, 1.1, 0.5)

# Grid search with cross-validation
grid = {
    'nlsvc__C': nlC_list,
    'nlsvc__gamma': gamma_list
}
grid_cv = GridSearchCV(pipe_SVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
nlsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_nlsvc__C']),
    ('gamma', grid_cv.cv_results_['param_nlsvc__gamma']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

nlsvc_score
# we can refine the range for gamma and broaden the range for C

C  gamma  mean accuracy  standard deviation
0    100  0.001        0.80700            0.002693
1    100  0.501        0.33800            0.006548
2    100  1.001        0.25350            0.002151
3   5100  0.001        0.80750            0.003062
4   5100  0.501        0.33775            0.007000
5   5100  1.001        0.25350            0.002151
6  10100  0.001        0.80600            0.003298
7  10100  0.501        0.33750            0.008292
8  10100  1.001        0.25350            0.002151

In [13]:
# let's define the values of parameters for grid search
nlC_list = np.arange(100, 11000, 5000)
gamma_list = np.arange(0.001, 0.11, 0.05)

# Grid search with cross-validation
grid = {
    'nlsvc__C': nlC_list,
    'nlsvc__gamma': gamma_list
}
grid_cv = GridSearchCV(pipe_SVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
nlsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_nlsvc__C']),
    ('gamma', grid_cv.cv_results_['param_nlsvc__gamma']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

nlsvc_score
# we can refine the range for gamma

C  gamma  mean accuracy  standard deviation
0    100  0.001        0.80750            0.003791
1    100  0.051        0.25150            0.000500
2    100  0.101        0.25025            0.000500
3   5100  0.001        0.80750            0.002739
4   5100  0.051        0.25150            0.000500
5   5100  0.101        0.25025            0.000500
6  10100  0.001        0.80700            0.002449
7  10100  0.051        0.25150            0.000500
8  10100  0.101        0.25025            0.000500

In [14]:
# let's define the values of parameters for grid search
nlC_list = np.arange(100, 11000, 5000)
gamma_list = np.arange(0.0001, 0.011, 0.005)

# Grid search with cross-validation
grid = {
    'nlsvc__C': nlC_list,
    'nlsvc__gamma': gamma_list
}
grid_cv = GridSearchCV(pipe_SVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
nlsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_nlsvc__C']),
    ('gamma', grid_cv.cv_results_['param_nlsvc__gamma']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

nlsvc_score
# we can refine the range for gamma

C   gamma  mean accuracy  standard deviation
0    100  0.0001        0.81650            0.012130
1    100  0.0051        0.49875            0.025335
2    100  0.0101        0.31375            0.023238
3   5100  0.0001        0.81800            0.014929
4   5100  0.0051        0.49925            0.025600
5   5100  0.0101        0.31450            0.024630
6  10100  0.0001        0.81575            0.011662
7  10100  0.0051        0.49875            0.024736
8  10100  0.0101        0.31475            0.025780

In [15]:
# let's define the values of parameters for grid search
nlC_list = np.arange(10, 1100, 500)
gamma_list = np.arange(0.00001, 0.0011, 0.0005)

# Grid search with cross-validation
grid = {
    'nlsvc__C': nlC_list,
    'nlsvc__gamma': gamma_list
}
grid_cv = GridSearchCV(pipe_SVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
nlsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_nlsvc__C']),
    ('gamma', grid_cv.cv_results_['param_nlsvc__gamma']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

nlsvc_score
# we can refine the range for gamma

C    gamma  mean accuracy  standard deviation
0    10    1e-05        0.82400            0.009533
1    10  0.00051        0.83300            0.006305
2    10  0.00101        0.80600            0.001837
3   510    1e-05        0.82250            0.013463
4   510  0.00051        0.83300            0.005624
5   510  0.00101        0.80525            0.002000
6  1010    1e-05        0.81525            0.012258
7  1010  0.00051        0.83300            0.006828
8  1010  0.00101        0.80650            0.002669

In [16]:
# let's define the values of parameters for grid search
nlC_list = np.arange(10, 1100, 100)
gamma_list = np.arange(0.00002, 0.001, 0.0002)

# Grid search with cross-validation
grid = {
    'nlsvc__C': nlC_list,
    'nlsvc__gamma': gamma_list
}
grid_cv = GridSearchCV(pipe_SVC, grid, cv=5, n_jobs = 3)

# Fit it to train data
grid_cv.fit(X_tr, y_tr)

# Collect results in a DataFrame
nlsvc_score = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_nlsvc__C']),
    ('gamma', grid_cv.cv_results_['param_nlsvc__gamma']),
    ('mean accuracy', grid_cv.cv_results_['mean_test_score']),
    ('standard deviation', grid_cv.cv_results_['std_test_score'])
])

nlsvc_score

C    gamma  mean accuracy  standard deviation
0     10    2e-05        0.83250            0.012425
1     10  0.00022        0.83175            0.010446
2     10  0.00042        0.83350            0.008155
3     10  0.00062        0.83075            0.007098
4     10  0.00082        0.81925            0.008860
5    110    2e-05        0.83075            0.015584
6    110  0.00022        0.82875            0.012273
7    110  0.00042        0.83475            0.006865
8    110  0.00062        0.83000            0.004472
9    110  0.00082        0.81825            0.009069
10   210    2e-05        0.82325            0.016948
11   210  0.00022        0.82900            0.013748
12   210  0.00042        0.83475            0.008639
13   210  0.00062        0.82950            0.005624
14   210  0.00082        0.81925            0.009035
15   310    2e-05        0.82425            0.015423
16   310  0.00022        0.82850            0.014392
17   310  0.00042        0.83300            0.005895
18   310  0.00062        0.82875            0.006124
19   310  0.00082        0.81825            0.009138
20   410    2e-05        0.82025            0.009981
21   410  0.00022        0.82875            0.012550
22   410  0.00042        0.83350            0.007517
23   410  0.00062        0.83025            0.005557
24   410  0.00082        0.81800            0.009172
25   510    2e-05        0.81075            0.007969
26   510  0.00022        0.82825            0.013775
27   510  0.00042        0.83425            0.007969
28   510  0.00062        0.82950            0.005099
29   510  0.00082        0.81825            0.009766
30   610    2e-05        0.80725            0.010290
31   610  0.00022        0.82875            0.015104
32   610  0.00042        0.83275            0.007517
33   610  0.00062        0.83050            0.005037
34   610  0.00082        0.81850            0.009467
35   710    2e-05        0.80700            0.012364
36   710  0.00022        0.82700            0.013053
37   710  0.00042        0.83400            0.008639
38   710  0.00062        0.83075            0.004912
39   710  0.00082        0.81850            0.009533
40   810    2e-05        0.80850            0.013861
41   810  0.00022        0.82700            0.012981
42   810  0.00042        0.83325            0.008201
43   810  0.00062        0.83000            0.005755
44   810  0.00082        0.81850            0.009233
45   910    2e-05        0.81075            0.015096
46   910  0.00022        0.82875            0.013346
47   910  0.00042        0.83300            0.007525
48   910  0.00062        0.83075            0.006052
49   910  0.00082        0.81825            0.008754
50  1010    2e-05        0.80600            0.013861
51  1010  0.00022        0.82625            0.013323
52  1010  0.00042        0.83450            0.008162
53  1010  0.00062        0.82950            0.006052
54  1010  0.00082        0.81850            0.009301

In [17]:
# let's find the row id where the value in 'validation accuracy' column
lsvc_id = lsvc_score['mean accuracy'].idxmax()
print('Linear SVM - top accuracy across folds: ', lsvc_score['mean accuracy'].at[lsvc_id], ' (std: ', 
      lsvc_score['standard deviation'].at[lsvc_id], ') with C: ', lsvc_score['C'].at[lsvc_id])

nlsvc_id = nlsvc_score['mean accuracy'].idxmax()
print('RBF SVM - top accuracy across folds: ', nlsvc_score['mean accuracy'].at[nlsvc_id], ' (std: ', 
      nlsvc_score['standard deviation'].at[nlsvc_id], ') with C: ', nlsvc_score['C'].at[nlsvc_id],
     'and gamma: ', nlsvc_score['gamma'].at[nlsvc_id])

Linear SVM - top accuracy across folds:  0.83275  (std:  0.014777516706131638 ) with C:  0.01
RBF SVM - top accuracy across folds:  0.83475  (std:  0.006864765108872986 ) with C:  110 and gamma:  0.00042


**Q:** Evaluate and report the accuracy of your tuned estimators on the 1,000 points from the test set

In [18]:
# Insert optimum C parameter for Linear SVC
pipe_LinearSVC_tuned = Pipeline([
    ('pca', pca), # The number of components in PCA are adopted from the "Data exploration" section
    ('lsvc', LinearSVC(C = 0.01, random_state=0))
])


# Insert optimum C and gamma parameters for Linear SVC
pipe_SVC_tuned = Pipeline([
    ('pca', pca), # The number of components in PCA are adopted from the "Data exploration" section
    ('nlsvc', SVC(C = 110, gamma = 0.00042, kernel='rbf', random_state=0))
])

# Fit the tuned pipelines in the entire train set
pipe_LinearSVC_tuned.fit(X_tr, y_tr)
pipe_SVC_tuned.fit(X_tr, y_tr)

# Save accuracy on test set
LinearSVC_accuracy = pipe_LinearSVC_tuned.score(X_te, y_te)
SVC_accuracy = pipe_SVC_tuned.score(X_te, y_te)

#let's print the accuracies of the test set
print('Linear SVM accuracy (test set): ', LinearSVC_accuracy)
print('RBF SVM accuracy (test set): ', SVC_accuracy)

Linear SVM accuracy (test set):  0.803
RBF SVM accuracy (test set):  0.825


In [19]:
# Put the decision tree accuracy in a DataFrame to be saved for FINAL COMPARISON step
linSVM_df = pd.DataFrame({'model' : ['svm linear'], 'test accuracy' : [LinearSVC_accuracy]})
linSVM_df.to_csv('svm linear.csv')

# Put the random forest accuracy in a DataFrame to be saved for FINAL COMPARISON step
rbfSVM_df = pd.DataFrame({'model' : ['svm rbf'], 'test accuracy' : [SVC_accuracy]})
rbfSVM_df.to_csv('svm rbf.csv')